In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact
import ipywidgets as widgets

@interact
def non_linear(indeks=0):
    G_konstanta = 6.67e-11 # SI (m^3 kg^-1 s^-2)

    # parameter model sintetik
    x0 = 400 # m
    z0 = 300 # m
    R = 100 # m
    rho = 3000 # kg/m3

    # variabel bebas x
    x = np.arange(0,1001,50)

    # fungsi forward modelling bola homogen
    def f(x0, z0, R, rho):
        return G_konstanta*((4/3)*np.pi*(R**3)*z0*rho)/(((x-x0)**2+(z0**2))**(3/2))*10e5

    # matriks kernel
    G = np.column_stack([f(x0, z0, R, rho)])

    # fungsi bola sphere
    def bola_sphere(x0, z0, R):
        q = np.linspace(0,2*np.pi,1000)

        xSphere = R*np.cos(q)+x0
        zSphere = R*np.sin(q)+z0

        return xSphere, zSphere

    # data observasi sintetik
    d_obs = f(x0, z0, R, rho)

    # model referensi
    m_ref = [100,100] # x0,z0

    banyak_iterasi = 100
    misfit = np.zeros(banyak_iterasi)

    # inversi non linier
    for i in range(indeks):
        i = indeks
        # forward modelling
        d_fm = f(m_ref[0],m_ref[1],R,rho)
        delta_d = d_obs - d_fm

        # menghitung misfit
        misfit[indeks] = np.sum(delta_d**2)

        if misfit[indeks] == 0 or misfit[indeks-1] == misfit[i]:
            break
        else:
            # matrix jacobi
            turunanx0 = G_konstanta*(4/3)*np.pi*(R**3)*m_ref[1]*rho*2*(x-m_ref[0])/(((x-m_ref[0])**2 + (0-m_ref[1])**2)**(5/2))*10e5
            turunanz0 = G_konstanta*(4/3)*np.pi*(R**3)*m_ref[1]*rho*2*(0-m_ref[1])/(((x-m_ref[0])**2 + (0-m_ref[1])**2)**(5/2))*10e5
            J = np.column_stack([turunanx0, turunanz0])

            # inversi
            m_calc = m_ref + np.linalg.inv((J.T).dot(J)).dot(J.T).dot(delta_d)
            m_ref = m_calc
    
    xb,yb = bola_sphere(x0, z0, R)
    xb1,yb1 = bola_sphere(m_ref[0], m_ref[1], R)
    d_fm = f(m_ref[0],m_ref[1],R,rho)
    
    # plotting
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex = True, figsize=(6,6))

    ax1.plot(x,d_obs,'ko', label="data observasi")
    ax1.plot(x,d_fm,'r-', label="data kalkulasi")
    ax1.legend()
    ax1.set_ylim(0, 10)
    ax1.set_xlim(-100,1050)
    ax1.set_ylabel("anomali gravitasi (mGal)")
    ax1.grid()
    
    ax2.plot(xb,yb,'k')
    ax2.plot(xb1,yb1,'r')
    ax2.set_ylim(-50,500)
    ax2.set_xlim(-100,1050)
    ax2.grid()
    ax2.set_ylabel("kedalaman (m)")
    ax2.set_xlabel("jarak (m)")
    ax2.annotate(f"iterasi= {indeks}", xy=[800, 100])
    ax2.annotate(f"RMSE= {misfit[indeks]}", xy=[800, 200])
    ax2.invert_yaxis()
    plt.show()

interactive(children=(IntSlider(value=0, description='indeks', max=1), Output()), _dom_classes=('widget-intera…

In [2]:
interact(non_linear, indeks=widgets.Play(min=0,max=20, interval=1000))
plt.show()

interactive(children=(Play(value=0, description='indeks', interval=1000, max=20), Output()), _dom_classes=('wi…

In [3]:
10e-5

0.0001